In [1]:
# Librarys
%run ForeSignalScraper.ipynb function(CustomizeFoSignals)
%run OpenOrdersMT5.ipynb function(Expiration_time,Determination_Set_Order)
%run sqlite3.ipynb function(Create_Database_Table,Insert_Data)
from datetime import datetime
from collections import OrderedDict
import random
import time

In [2]:
def ConsolidationResults (Currencyslist,desired_time,FoWebsiteurl,Foclassorid,Foclassoridvalue,ForeSignalDriver,final_list,
                         DatabaseName,TableName):
                         
    # Empty Dicts and Lists
    final_informations = OrderedDict()
    History = OrderedDict()
    FoFinalList = list()
    Reference_list = list()
    Order_List = list()
    History_counter = 1
    Create_Database_Table(DatabaseName,TableName)
    
    while True:
    
        # Get Datas From Fore Signal. If lengh data > 10 and not in FoFinalList, data appended into FoFinalList
        
        FoFinalList_start_len = len(FoFinalList)
        
        for items in CustomizeFoSignals(ForeSignalDriver,FoWebsite_url,Foclassorid,Foclassoridvalue):
            if len(items) > 10 and items not in FoFinalList:
                FoFinalList.append(items)
                
        FoFinalList_Final_len = len(FoFinalList)
            
        if FoFinalList_start_len < FoFinalList_Final_len:    
    
            # Get Current Price Every time and Update it
    
            Tradingview_price_dictionary = OrderedDict()
        
            for item in range (0,9):
                now = datetime.now()
                final_list[0].switch_to.window(final_list[0].window_handles[item])
                current_date = f'{Currencys_list[item]} : {now}'
                current_price = final_list[1][item].text
                Tradingview_price_dictionary[current_date] = current_price
        
#             Merge Foresignal and Current Price of Tradingvirw Datas in a Dictionary
        
            for item_fo in FoFinalList:
                for k,v in Tradingview_price_dictionary.items():
                    if item_fo[0] in k[:6]:
                        item_fo.append(k)
                        item_fo.append(v)
                        final_informations[f'{item_fo[:11]}'] = item_fo
    
            Reference_list_start_len = len(Reference_list)
            Reference_list = list(final_informations.items()) 
            Reference_list_final_len = len(Reference_list)
    
            if Reference_list_start_len < Reference_list_final_len:
        
                    for items in Reference_list[Reference_list_start_len:Reference_list_final_len]:
                        Order_List.append(items)
                
        for items in Order_List:
            
            if len(items[1])>13:
                items[1].remove(items[1][11])
                items[1].remove(items[1][11])
        
            current_time = str(datetime.now()).split()[1].replace(':',' ').split()  
            Order_hour = int(items[1][2].replace(':',' ').split()[0])    
            Order_minute = int(items[1][2].replace(':',' ').split()[1])
                    
            if int(current_time[0]) >= Order_hour:
            
                Symbole = items[1][0]
                Entry_Price = float(items[1][6])
                Takeprofit_Price = float(items[1][8])
                Stoploss_Price = float(items[1][10])
                Current_Price = float(items[1][12])
                current_time = items[1][4].replace(':',' ').split()
                expiration = Expiration_time (int(current_time[0]),int(current_time[1]))
                    
                if items[1][5] == 'Buy at' and Entry_Price > Current_Price:
                    Type_of_Trade = 'buy stop'
#                         Stoploss_Price = float(items[1][10])+((Entry_Price-float(items[1][10]))/2)
                
                elif items[1][5] == 'Buy at' and Entry_Price < Current_Price:
                    Type_of_Trade = 'buy limit'
#                         Stoploss_Price = float(items[1][10])+((Entry_Price-float(items[1][10]))/2)
            
                elif items[1][5] == 'Sell at' and Entry_Price < Current_Price:
                    Type_of_Trade = 'sell stop'
#                         Stoploss_Price = float(items[1][10])-((float(items[1][10])- Entry_Price)/2)
            
                elif items[1][5] == 'Sell at' and Entry_Price > Current_Price:
                    Type_of_Trade = 'sell limit'
#                         Stoploss_Price = float(items[1][10])-((float(items[1][10])- Entry_Price)/2)
                        
                request = Determination_Set_Order(Type_of_Trade, Symbole , 0.05 ,Entry_Price
                                              ,Stoploss_Price,Takeprofit_Price,expiration)   
                result = mt.order_send(request)
                result
                                        
#                     TableName = 'Informations'
#                     InfoTuple = (History_counter, items[1][0], items[1][1], items[1][2], items[1][3], items[1][4],items[1][5]
#                                      , float(items[1][6]), float(items[1][8]), float(items[1][10]), items[1][11][8:], float(items[1][12]))
                                 
#                     History_counter += 1
#                     send_datas(conn_str,TableName,InfoTuple)
                
                InfoTuple = (items[1][0], items[1][1], items[1][2], items[1][3], items[1][4], items[1][5], float(items[1][6])
                                     , float(items[1][8]), float(items[1][10]), items[1][11][8:], float(items[1][12]))
        
                Insert_Data (DatabaseName,TableName,InfoTuple)
                
                Order_List.remove(items)
            
            if len(items[1])>13:
                items[1].remove(items[1][11])
                items[1].remove(items[1][11])
        
        delay_time = random.randrange(0,desired_time)
        time.sleep(delay_time)